In [1]:
# Environmental Data is Calculated and Stored in a Database

In [2]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, mapping, Point, box
import rasterio
from rasterio.features import shapes
from rasterio.mask import mask
from scipy.stats import mode
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from shapely.ops import unary_union
import os


In [3]:
# Dictionary mapping landcover codes to names
landcover_names = {
    10: 'Tree cover',
    20: 'Shrubland',
    30: 'Grassland',
    40: 'Cropland',
    50: 'Built-up',
    60: 'Bare / sparse vegetation',
    70: 'Snow and Ice',
    80: 'Permanent water bodies',
    90: 'Herbaceous wetland',
    95: 'Mangroves',
    100: 'Moss and lichen'
}


def calculate_landcover(row, landcover_tif):
    with rasterio.open(landcover_tif) as src:
        # Mask the raster with the geometry from the row and crop to the minimal enclosing rectangle
        lc_data, _ = mask(src, [row.geometry], crop=True)
    
    # Filter out values outside the range [10, 100] and treat them as NaNs
    lc_data = np.where((lc_data > 100) | (lc_data < 10), np.nan, lc_data)
    
    # Flatten the array to one dimension and remove NaN values
    lc_values = lc_data.flatten()
    lc_values = lc_values[~np.isnan(lc_values)]
    
    # Calculate the mode of the valid land cover values
    if lc_values.size > 0:
        # Find unique values and their corresponding frequencies
        unique, counts = np.unique(lc_values, return_counts=True)
        # Find the index of the maximum count
        max_index = np.argmax(counts)
        # The most frequent value
        mode_value = unique[max_index]
    else:
        mode_value = np.nan  # Return NaN if there are no valid values
    
    return mode_value


def calculate_vegheight(row, vegheight_tif):
    with rasterio.open(vegheight_tif) as src:
        vg_data, _ = mask(src, [row.geometry], crop=True)
    
    vg_data = np.where((vg_data > 100) | (vg_data < 10), np.nan, vg_data)
    
    vg_values = vg_data.flatten()
    vg_values = vg_values[~np.isnan(vg_values)]  
    
    mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
    
    return mean_vg

def get_landcover(shapes, landcover_tif):
    with rasterio.open(landcover_tif) as src:
        bounds = src.bounds
        bounding_box = box(bounds[0], bounds[1], bounds[2], bounds[3])

    for index, row in shapes.iterrows():
        if row.geometry.intersects(bounding_box):
            lc_value = calculate_landcover(row, landcover_tif)
            if not np.isnan(lc_value):
                lc_name = landcover_names.get(int(lc_value), 'Unknown')  # Get the corresponding name from the dictionary

                # Save the value in the dataframe
                shapes.at[index, 'landcover_type'] = lc_name
            else:
                shapes.at[index, 'landcover_type'] = 'Unknown'

    return shapes

def get_veg_height(shapes, vegheight_tif):
    with rasterio.open(vegheight_tif) as src:
        bounds = src.bounds
        bounding_box = box(bounds[0], bounds[1], bounds[2], bounds[3])

    for index, row in shapes.iterrows():
        if row.geometry.intersects(bounding_box):
            vh_value = calculate_vegheight(row, vegheight_tif)  # Use calculate_vegheight function
            if not np.isnan(vh_value):

                # Save the value in the dataframe
                shapes.at[index, 'vegetation_height'] = vh_value  # Save in 'vegetation_height' column
            else:
                shapes.at[index, 'vegetation_height'] = np.nan  # Handle NaN values if needed

    return shapes

In [4]:
regions = ['herschel', 'peel', 'kolguev', 'gydan']
types = ['DEM', 'OP', 'OT']

for region in regions:
    for type in types:
        input_filepath = f'Results\\all\\{region}_{type}_labels_results.geojson'
        output_filepath = f'Results\\all\\{region}_{type}_labels_results.geojson'
        landcover_tif = f'environmental\\land_cover_2020\\{region}_rp_land_cover_2020.tif'

        # Read GeoJSON file
        shapes = gpd.read_file(input_filepath)

        # Calculate mean slope angle for each object
        shapes = get_landcover(shapes, landcover_tif)

        # Save the modified GeoJSON file
        shapes.to_file(output_filepath, driver='GeoJSON')
        shapes.to_file(f'Results\\{region}\\{region}_{type}_labels_results.geojson', driver='GeoJSON')


herschel
DEM
OP
OT
peel
DEM
OP
OT
kolguev
DEM
OP
OT
gydan
DEM
OP
OT


In [5]:
regions = ['herschel', 'gydan']
types = ['DEM', 'OP', 'OT']

for region in regions:
    print(region)
    for type in types:
        print(type)
        input_filepath = f'Results\\all\\{region}_{type}_labels_results.geojson'
        output_filepath = f'Results\\all\\{region}_{type}_labels_results.geojson'
        vegheight_tif = f'environmental\\vegetation_height\\{region}_rp_veg_height.tif'

        # Read GeoJSON file
        shapes = gpd.read_file(input_filepath)

        # Calculate mean slope angle for each object
        shapes = get_veg_height(shapes, vegheight_tif)

        # Save the modified GeoJSON file
        shapes.to_file(output_filepath, driver='GeoJSON')
        shapes.to_file(f'Results\\{region}\\{region}_{type}_labels_results.geojson', driver='GeoJSON')

herschel
DEM


C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nan

OP


C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nan

OT


...

C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values


OP


C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nan

OT


C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nanmean(vg_values)  # Using nanmedian to handle NaN values
C:\Users\Noe\AppData\Local\Temp\ipykernel_19892\2247354678.py:62: RuntimeWarning: Mean of empty slice
  mean_vg = np.nan